In [306]:
import pandas as pd
import numpy as np
import itertools

In [434]:
def compute_the_thing(input_file="test_input.txt"):
    
    with open(input_file, "r") as f:
        data = f.readlines()
        
    data = [line.strip("\n") for line in data]
    
    caves = []
    for line in data:
        i,j = line.split("-")
        caves.extend([i,j])
    caves = list(set(caves))
    #print("unique caves: ", caves)
    
    #print("Checking middles...")
    valid_data = data.copy()
    for cave in caves:
        if (cave != "start") and (cave != "end"):
            paths_with_cave = [line for line in data if (cave in line) and ("end" not in line) and ("start" not in line)]
            #print(cave, paths_with_cave)
            if len(paths_with_cave) == 1:
                i,j = paths_with_cave[0].split("-")
                if (i != cave) and (i == i.lower()):
                    valid_data.remove(paths_with_cave[0])
                    #print("removing: ", paths_with_cave[0])
                elif (j != cave) and (j == j.lower()):
                    valid_data.remove(paths_with_cave[0])
                    #print("removing: ", paths_with_cave[0])
    #print()
    valid_caves = []
    for line in valid_data:
        i,j = line.split("-")
        valid_caves.extend([i,j])
    valid_caves = list(set(valid_caves))
    #print("valid caves: ", valid_caves)
    
    small_caves = [cave for cave in valid_caves if (cave==cave.lower()) and (cave!="start") and (cave!="end")]
    large_caves = [cave for cave in valid_caves if (cave==cave.upper())]
    #print("small", small_caves)
    #print("large", large_caves)
    #print()
    
    starts = [line for line in valid_data if "start" in line]
    things_to_replace = []
    for start,idx in zip(starts,range(len(starts))):
        i,j = start.split("-")
        if i != "start":
            things_to_replace.append((idx,"-".join([j,i])))
    if len(things_to_replace) > 0:
        for thing in things_to_replace:
            idx, value = thing
            starts[idx] = value
            
    middles = [line for line in valid_data if ("start" not in line) and ("end" not in line)]
    reversed_middles = []
    for middle in middles:
        i,j = middle.split("-")
        reversed_middles.append("-".join([j,i]))
    middles.extend(reversed_middles)
    
    ends = [line for line in valid_data if "end" in line]
    things_to_replace = []
    for end,idx in zip(ends,range(len(ends))):
        i,j = end.split("-")
        if j != "end":
            things_to_replace.append((idx,"-".join([j,i])))
    if len(things_to_replace) > 0:
        for thing in things_to_replace:
            idx, value = thing
            ends[idx] = value
    
    #print("starts: ", starts)
    #print("middles: ", middles)
    #print("ends: ", ends)
    
    middles_ends = []
    middles_ends.extend(middles)
    middles_ends.extend(ends)
    #print()
    finished_paths = []
    unfinished_paths = []
    for start in starts:
        next_path = start.split("-")[1]
        # print(start,next_path)
        possible_paths = [path for path in middles_ends if path.split("-")[0]==next_path]
        # print(possible_paths)
        for path in possible_paths:
            if "end" in path:
                finished_paths.append([start.split("-")[0],start.split("-")[1],path.split("-")[1]])
            else:
                unfinished_paths.append([start.split("-")[0],start.split("-")[1],path.split("-")[1]])
    # print("finished: ", len(finished_paths), finished_paths)   
    # print("unfinished: ", len(unfinished_paths), unfinished_paths)
    i = 0
    while len(unfinished_paths) > 0:
        #print()
        #print(i)
        unfinished = unfinished_paths.pop()
        next_node = unfinished[-1]
        #print(unfinished, next_node)
        possible_paths = [path for path in middles_ends 
                          if (path.split("-")[0]==next_node) and (path not in unfinished)]
        #print(unfinished, next_node,possible_paths)
        for path in possible_paths:
            #print()
            #print(i, "possible path", unfinished, path)
            temp = unfinished.copy()
            temp.append(path.split("-")[1])
            if "end" in path:
                finished_paths.append(temp)
                #print(i, "finished: ", temp)
            else:
                # check small caves
                valid_path = False
                for cave in small_caves:
                    if cave in path:
                        if sum([1 for p in temp if p==cave]) > 1:
                            # print(i, "invalid path", temp)
                            continue
                        elif (temp not in finished_paths):
                            # print(i, cave, "unfinished: ", temp)
                            unfinished_paths.append(temp)
        #print()
        #print(i,"finished: ", len(finished_paths))
        #for path in finished_paths:
        #    print(path)
        #print(i,"unfinished: ", len(unfinished_paths))
        #for path in unfinished_paths:
        #    print(path)
        i += 1
        #if i > 10:
        #    break
    # check small caves again
    #print("checking small caves again...")
    finished_paths = [tuple(path) for path in finished_paths]
    finished_paths = list(set(finished_paths))
    final_paths = [tuple(path) for path in finished_paths]
    
    for path,idx in zip(finished_paths, range(len(finished_paths))):
        for cave in small_caves:
            if cave in path:
                #print(path,cave)
                if sum([1 for p in path if p==cave]) > 1:
                    # print(sum([1 for p in path if p==cave]), "invalid path", path)
                    if path in final_paths:
                        final_paths.remove(path)

    final_paths = list(set(final_paths))
    #print()
    #print("final: ", len(finished_paths), len(final_paths), print(len(set(final_paths))))
    #for path in sorted(final_paths):
    #    print(path)
    #print("unfinished: ", len(unfinished_paths))
    #for path in unfinished_paths:
    #print(path)
        
    num_paths = len(final_paths)
    return data, final_paths, num_paths

In [435]:
data, final_paths, num_paths = compute_the_thing(input_file="test_input_small.txt")

In [436]:
assert num_paths == 10

In [437]:
data, final_paths, num_paths = compute_the_thing(input_file="test_input_medium.txt")

In [438]:
assert num_paths == 19

In [439]:
data, final_paths, num_paths = compute_the_thing(input_file="test_input_large.txt")

In [440]:
assert num_paths == 226

In [441]:
data, final_paths, num_paths = compute_the_thing(input_file="input.txt")

In [442]:
num_paths

3497

In [509]:
def compute_the_thing2(input_file="test_input.txt", allowed_smalls=1):
    
    with open(input_file, "r") as f:
        data = f.readlines()
        
    data = [line.strip("\n") for line in data]
    
    caves = []
    for line in data:
        i,j = line.split("-")
        caves.extend([i,j])
    caves = list(set(caves))
    #print("unique caves: ", caves)
    
    #print("Checking middles...")
    valid_data = data.copy()
    for cave in caves:
        if (cave != "start") and (cave != "end"):
            paths_with_cave = [line for line in data if (cave in line) and ("end" not in line) and ("start" not in line)]
            #print(cave, paths_with_cave)
            if len(paths_with_cave) == 1:
                i,j = paths_with_cave[0].split("-")
                if (i != cave) and (i == i.lower()):
                    valid_data.remove(paths_with_cave[0])
                    #print("removing: ", paths_with_cave[0])
                elif (j != cave) and (j == j.lower()):
                    valid_data.remove(paths_with_cave[0])
                    #print("removing: ", paths_with_cave[0])
    #print()
    valid_caves = []
    for line in valid_data:
        i,j = line.split("-")
        valid_caves.extend([i,j])
    valid_caves = list(set(valid_caves))
    #print("valid caves: ", valid_caves)
    
    small_caves = [cave for cave in valid_caves if (cave==cave.lower()) and (cave!="start") and (cave!="end")]
    large_caves = [cave for cave in valid_caves if (cave==cave.upper())]
    #print("small", small_caves)
    #print("large", large_caves)
    #print()
    
    starts = [line for line in valid_data if "start" in line]
    things_to_replace = []
    for start,idx in zip(starts,range(len(starts))):
        i,j = start.split("-")
        if i != "start":
            things_to_replace.append((idx,"-".join([j,i])))
    if len(things_to_replace) > 0:
        for thing in things_to_replace:
            idx, value = thing
            starts[idx] = value
            
    middles = [line for line in valid_data if ("start" not in line) and ("end" not in line)]
    reversed_middles = []
    for middle in middles:
        i,j = middle.split("-")
        reversed_middles.append("-".join([j,i]))
    middles.extend(reversed_middles)
    
    ends = [line for line in valid_data if "end" in line]
    things_to_replace = []
    for end,idx in zip(ends,range(len(ends))):
        i,j = end.split("-")
        if j != "end":
            things_to_replace.append((idx,"-".join([j,i])))
    if len(things_to_replace) > 0:
        for thing in things_to_replace:
            idx, value = thing
            ends[idx] = value
    
    #print("starts: ", starts)
    #print("middles: ", middles)
    #print("ends: ", ends)
    
    middles_ends = []
    middles_ends.extend(middles)
    middles_ends.extend(ends)
    #print()
    finished_paths = []
    unfinished_paths = []
    for start in starts:
        next_path = start.split("-")[1]
        # print(start,next_path)
        possible_paths = [path for path in middles_ends if path.split("-")[0]==next_path]
        # print(possible_paths)
        for path in possible_paths:
            if "end" in path:
                finished_paths.append([start.split("-")[0],start.split("-")[1],path.split("-")[1]])
            else:
                unfinished_paths.append([start.split("-")[0],start.split("-")[1],path.split("-")[1]])
    # print("finished: ", len(finished_paths), finished_paths)   
    # print("unfinished: ", len(unfinished_paths), unfinished_paths)
    i = 0
    while len(unfinished_paths) > 0:
        #print()
        #print(i)
        unfinished = unfinished_paths.pop(0)
        next_node = unfinished[-1]
        #print(unfinished, next_node)
        possible_paths = [path for path in middles_ends 
                          if (path.split("-")[0]==next_node) and (path not in unfinished)]
        #print(unfinished, next_node,possible_paths)
        for path in possible_paths:
            temp = unfinished.copy()
            temp.append(path.split("-")[1])
            # print()
            # print(i, "possible path", temp)
            for cave in small_caves:
                if cave in temp:
                    if temp.count(cave) <= allowed_smalls:
                        if "end" in temp and temp not in finished_paths:
                            finished_paths.append(temp)
                        elif temp not in unfinished_paths:
                            unfinished_paths.append(temp)
                            print(len(unfinished_paths), "unfinished paths remaining")
        #print()
        #print(i,"finished: ", len(finished_paths))
        #for path in finished_paths:
        #    print(path)
        #print(i,"unfinished: ", len(unfinished_paths))
        #for path in unfinished_paths:
        #    print(path)
        i += 1
        # print(i)
        if i > 40:
            break
    # check small caves again
    #print("checking small caves again...")
    finished_paths = [tuple(path) for path in finished_paths]
    finished_paths = list(set(finished_paths))
    final_paths = [tuple(path) for path in finished_paths]
    
    for path,idx in zip(finished_paths, range(len(finished_paths))):
        for cave in small_caves:
            if cave in path:
                #print(path,cave)
                if sum([1 for p in path if p==cave]) > allowed_smalls:
                    # print(sum([1 for p in path if p==cave]), "invalid path", path)
                    if path in final_paths:
                        final_paths.remove(path)
    
    final_paths = list(set(final_paths))
    #print()
    print("final: ", len(finished_paths), len(final_paths), print(len(set(final_paths))))
    #for path in sorted(final_paths):
    #    print(path)
    #print("unfinished: ", len(unfinished_paths))
    #for path in unfinished_paths:
    #print(path)
        
    num_paths = len(final_paths)
    return data, final_paths, num_paths

In [510]:
data, final_paths, num_paths = compute_the_thing2(input_file="test_input_small.txt", allowed_smalls=1)
assert num_paths == 10

3 unfinished paths remaining
3 unfinished paths remaining
3 unfinished paths remaining
3 unfinished paths remaining
3 unfinished paths remaining
3 unfinished paths remaining
3 unfinished paths remaining
4 unfinished paths remaining
4 unfinished paths remaining
4 unfinished paths remaining
5 unfinished paths remaining
6 unfinished paths remaining
6 unfinished paths remaining
7 unfinished paths remaining
8 unfinished paths remaining
7 unfinished paths remaining
8 unfinished paths remaining
9 unfinished paths remaining
9 unfinished paths remaining
9 unfinished paths remaining
8 unfinished paths remaining
8 unfinished paths remaining
7 unfinished paths remaining
7 unfinished paths remaining
6 unfinished paths remaining
6 unfinished paths remaining
6 unfinished paths remaining
6 unfinished paths remaining
6 unfinished paths remaining
6 unfinished paths remaining
6 unfinished paths remaining
6 unfinished paths remaining
6 unfinished paths remaining
6 unfinished paths remaining
6 unfinished p

In [491]:
num_paths

10

In [492]:
final_paths

[('start', 'b', 'end'),
 ('start', 'A', 'c', 'A', 'b', 'A', 'end'),
 ('start', 'A', 'c', 'A', 'b', 'end'),
 ('start', 'A', 'c', 'A', 'end'),
 ('start', 'A', 'b', 'end'),
 ('start', 'b', 'A', 'c', 'A', 'end'),
 ('start', 'A', 'b', 'A', 'c', 'A', 'end'),
 ('start', 'A', 'end'),
 ('start', 'A', 'b', 'A', 'end'),
 ('start', 'b', 'A', 'end')]